# pip & import

In [31]:
pip install pandas opencv-python numpy scikit-image scipy scikit-learn matplotlib tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import pandas as pd
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern, graycomatrix, graycoprops
from skimage.measure import moments_hu
from scipy.stats import skew, entropy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

# CSV generation

In [ ]:
import os
import shutil
import csv

# Input and output folders
input_folder = 'images'
output_folder = 'output_images'
csv_file_path = 'image_classes.csv'

# Only handle these classes
classes = {'1', '2', '3'}

# Create output subfolders
for cls in classes:
    os.makedirs(os.path.join(output_folder, cls), exist_ok=True)

# Collect CSV rows
csv_rows = [('class', 'path')]

# Process files
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        try:
            # Assumed filename format: name_class.png
            class_part = filename.rsplit('_', 1)[-1].split('.')[0]
            if class_part in classes:
                src = os.path.join(input_folder, filename)
                dst = os.path.join(output_folder, class_part, filename)
                shutil.copy(src, dst)

                # Write relative path to CSV
                relative_path = os.path.join(output_folder, class_part, filename)
                csv_rows.append((class_part, relative_path))
            else:
                print(f"Skipped (unknown class): {filename}")
        except Exception as e:
            print(f"Error: {filename} – {e}")

# Save CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(csv_rows)

print(f"CSV file created: {csv_file_path}")


CSV fájl elkészült: kep_osztalyok.csv


# Preprocessing

## Setting images to a uniform resolution

In [ ]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Load CSV file
data = pd.read_csv('image_classes.csv')
image_paths = data['path'].values
labels = data['class'].values

# Load and resize images
def load_images(paths, target_size=(128, 128)):
    images = []
    for path in paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        img = cv2.resize(img, target_size)            # Resize
        images.append(img)
    return np.array(images)

images = load_images(image_paths)
clahe_images = images / 255.0  # Normalize to [0,1] range

## Noise filtering - Gaussian blur

In [11]:
def denoise_images(images):
    denoised = []
    for img in images:
        denoised_img = cv2.GaussianBlur(img, (5, 5), 0)
        denoised.append(denoised_img)
    return np.array(denoised)

## Contrast Enhancement - CLAHE (Contrast Limited Adaptive Histogram Equalization)

In [12]:
def apply_clahe(images):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_images = []
    for img in images:
        clahe_img = clahe.apply((img * 255).astype(np.uint8)) / 255.0
        clahe_images.append(clahe_img)
    return np.array(clahe_images)

## Edge highlighting

### Sobel

In [13]:
def sobel_edge_detection(images):
    edge_images = []
    for img in images:
        sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        sobel = np.sqrt(sobel_x**2 + sobel_y**2)
        edge_images.append(sobel / sobel.max())
    return np.array(edge_images)

### Canny

In [14]:
def canny_edge_detection(images, low_threshold=0.1, high_threshold=0.3):
    edge_images = []
    for img in images:
        img_uint8 = (img * 255).astype(np.uint8)
        edges = cv2.Canny(img_uint8, low_threshold * 255, high_threshold * 255)
        edge_images.append(edges / 255.0)
    return np.array(edge_images)

# Extract features

## HOG (Histogram of Oriented Gradients)

In [15]:
def extract_hog_features(images):
    hog_features = []
    for img in images:
        features, _ = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
        hog_features.append(features)
    return np.array(hog_features)

## LBP (Local Binary Patterns)

In [16]:
def extract_lbp_features(images, radius=3, n_points=24):
    lbp_features = []
    for img in images:
        lbp = local_binary_pattern(img, n_points, radius, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), density=True)
        lbp_features.append(hist)
    return np.array(lbp_features)

## SIFT (Scale-Invariant Feature Transform)

In [17]:
def extract_sift_features(images, max_features=100):
    sift = cv2.SIFT_create()
    sift_features = []
    for img in images:
        img_uint8 = (img * 255).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(img_uint8, None)
        if descriptors is not None and len(descriptors) > 0:
            if len(descriptors) > max_features:
                descriptors = descriptors[:max_features]
            padded_descriptors = np.zeros((max_features, 128))
            padded_descriptors[:len(descriptors)] = descriptors
        else:
            padded_descriptors = np.zeros((max_features, 128))
        sift_features.append(padded_descriptors.flatten())
    return np.array(sift_features)

## GLCM (Gray-Level Co-occurrence Matrix)

In [18]:
def extract_glcm_features(images, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256):
    glcm_features = []
    for img in images:
        img_quantized = (img * (levels - 1)).astype(np.uint8)
        glcm = graycomatrix(img_quantized, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast').flatten()
        homogeneity = graycoprops(glcm, 'homogeneity').flatten()
        energy = graycoprops(glcm, 'energy').flatten()
        correlation = graycoprops(glcm, 'correlation').flatten()
        features = np.concatenate([contrast, homogeneity, energy, correlation])
        glcm_features.append(features)
    return np.array(glcm_features)

## Statistical features

In [19]:
def extract_statistical_features(images):
    stat_features = []
    for img in images:
        mean = np.mean(img)
        std = np.std(img)
        skewness = skew(img.flatten())
        hist, _ = np.histogram(img.flatten(), bins=256, range=[0, 1], density=True)
        img_entropy = entropy(hist)
        hu_moments = moments_hu(img)
        features = np.concatenate([[mean, std, skewness, img_entropy], hu_moments])
        stat_features.append(features)
    return np.array(stat_features)

# Analysis of variance and curve plotting

In [ ]:
def analyze_variance(features_scaled, feature_name, preprocess_name, feature_dir):
    pca = PCA()
    pca.fit(features_scaled)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    
    var_thresholds = [0.80, 0.90, 0.95]
    for threshold in var_thresholds:
        n_components = np.where(cumulative_variance >= threshold)[0][0] + 1
        print(f"{feature_name}_{preprocess_name}: {threshold*100}% variance is reached with {n_components} components.")
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance ratio')
    plt.title(f'{feature_name}_{preprocess_name} PCA explained variance')
    plt.grid(True)
    plt.savefig(os.path.join(feature_dir, f'{feature_name}_{preprocess_name}_variance_plot.png'))
    plt.close()
    print(f"{feature_name}_{preprocess_name}_variance_plot.png saved.")
    
    return cumulative_variance

# Save original features

In [ ]:
def save_original_to_csv(features, labels, feature_name, preprocess_name, output_dir):
    columns = [f'{feature_name}_{i}' for i in range(features.shape[1])] + ['class']
    df = pd.DataFrame(np.hstack((features, labels.reshape(-1, 1))), columns=columns)
    filename = os.path.join(output_dir, f'{feature_name}_{preprocess_name}_features.csv')
    df.to_csv(filename, index=False)
    print(f"{filename} saved.")

# PCA dimension reduction and saving

In [ ]:
def save_pca_to_csv(reduced_features, labels, feature_name, preprocess_name, feature_dir, n_components):
    columns = [f'pca_{i}' for i in range(n_components)] + ['class']
    df = pd.DataFrame(np.hstack((reduced_features, labels.reshape(-1, 1))), columns=columns)
    filename = os.path.join(feature_dir, f'{feature_name}_{preprocess_name}_pca_{n_components}.csv')
    df.to_csv(filename, index=False)
    print(f"{filename} saved.")

In [ ]:
def apply_pca_and_save(features, labels, feature_name, preprocess_name, output_dir, n_components_list):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    feature_dir = os.path.join(output_dir, feature_name)
    os.makedirs(feature_dir, exist_ok=True)
    
    analyze_variance(features_scaled, feature_name, preprocess_name, feature_dir)
    
    max_components = min(features.shape[0], features.shape[1])
    
    for n_components in n_components_list:
        if n_components > max_components:
            print(f"{feature_name}_{preprocess_name}: {n_components} components not possible, max {max_components}.")
            continue
        
        pca = PCA(n_components=n_components)
        reduced_features = pca.fit_transform(features_scaled)
        save_pca_to_csv(reduced_features, labels, feature_name, preprocess_name, feature_dir, n_components)

# Save original characteristics and PCA reduced characteristics

In [ ]:
# Create main directories
original_output_dir = 'original_features'
pca_output_dir = 'feature_reduction'
os.makedirs(original_output_dir, exist_ok=True)
os.makedirs(pca_output_dir, exist_ok=True)

In [ ]:
# Preprocessing types and images
preprocess_types = {
    'original': clahe_images,
    'gaussian': denoise_images(clahe_images),
    'clahe': apply_clahe(clahe_images),
    'sobel': sobel_edge_detection(clahe_images),
    'canny': canny_edge_detection(clahe_images)
}

In [ ]:
# List of component numbers
n_components_list = [10, 20, 50, 100, 200, 500, 1000]

In [ ]:
for preprocess_name, images in tqdm(preprocess_types.items(), desc="Preprocessing types"):
    print(f"Preprocessing: {preprocess_name}")
    
    # Feature extraction
    hog_features = extract_hog_features(images)
    lbp_features = extract_lbp_features(images)
    sift_features = extract_sift_features(images)
    glcm_features = extract_glcm_features(images)
    stat_features = extract_statistical_features(images)
    hog_lbp_features = np.hstack((hog_features, lbp_features))
    all_combined_features = np.hstack((hog_features, lbp_features, sift_features, glcm_features, stat_features))
    
    # Define feature sets
    feature_sets = [
        (hog_features, 'hog'),
        (lbp_features, 'lbp'),
        (sift_features, 'sift'),
        (glcm_features, 'glcm'),
        (stat_features, 'stat'),
        (hog_lbp_features, 'hog_lbp_combined'),
        (all_combined_features, 'all_combined')
    ]
    
    # Save original features
    original_preprocess_dir = os.path.join(original_output_dir, preprocess_name)
    os.makedirs(original_preprocess_dir, exist_ok=True)
    for features, feature_name in tqdm(feature_sets, desc=f"Saving original features for {preprocess_name}"):
        save_original_to_csv(features, labels, feature_name, preprocess_name, original_preprocess_dir)
    
    # PCA reduction and saving
    pca_preprocess_dir = os.path.join(pca_output_dir, preprocess_name)
    os.makedirs(pca_preprocess_dir, exist_ok=True)
    for features, feature_name in tqdm(feature_sets, desc=f"{preprocess_name} PCA reduction"):
        apply_pca_and_save(features, labels, feature_name, preprocess_name, pca_preprocess_dir, n_components_list)


Előfeldolgozási típusok:   0%|          | 0/5 [00:00<?, ?it/s]

Előfeldolgozás: original


c:\Users\misur\AppData\Local\Programs\Python\Python312\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


original_features\original\hog_original_features.csv mentve.
original_features\original\lbp_original_features.csv mentve.


original_features\original\sift_original_features.csv mentve.
original_features\original\glcm_original_features.csv mentve.
original_features\original\stat_original_features.csv mentve.


...

feature_reduction\canny\glcm\glcm_canny_pca_10.csv mentve.
glcm_canny: 20 komponens nem lehetséges, max 16.
glcm_canny: 50 komponens nem lehetséges, max 16.
glcm_canny: 100 komponens nem lehetséges, max 16.
glcm_canny: 200 komponens nem lehetséges, max 16.
glcm_canny: 500 komponens nem lehetséges, max 16.
glcm_canny: 1000 komponens nem lehetséges, max 16.
stat_canny: 80.0% variancia eléréséhez 3 komponens szükséges.
stat_canny: 90.0% variancia eléréséhez 4 komponens szükséges.
stat_canny: 95.0% variancia eléréséhez 4 komponens szükséges.
stat_canny_variance_plot.png mentve.


feature_reduction\canny\stat\stat_canny_pca_10.csv mentve.
stat_canny: 20 komponens nem lehetséges, max 11.
stat_canny: 50 komponens nem lehetséges, max 11.
stat_canny: 100 komponens nem lehetséges, max 11.
stat_canny: 200 komponens nem lehetséges, max 11.
stat_canny: 500 komponens nem lehetséges, max 11.
stat_canny: 1000 komponens nem lehetséges, max 11.
hog_lbp_combined_canny: 80.0% variancia eléréséhez 281 komponens szükséges.
hog_lbp_combined_canny: 90.0% variancia eléréséhez 497 komponens szükséges.
hog_lbp_combined_canny: 95.0% variancia eléréséhez 711 komponens szükséges.
hog_lbp_combined_canny_variance_plot.png mentve.
feature_reduction\canny\hog_lbp_combined\hog_lbp_combined_canny_pca_10.csv mentve.
feature_reduction\canny\hog_lbp_combined\hog_lbp_combined_canny_pca_20.csv mentve.
feature_reduction\canny\hog_lbp_combined\hog_lbp_combined_canny_pca_50.csv mentve.
feature_reduction\canny\hog_lbp_combined\hog_lbp_combined_canny_pca_100.csv mentve.
feature_reduction\canny\hog_lbp_

feature_reduction\canny\hog_lbp_combined\hog_lbp_combined_canny_pca_1000.csv mentve.
all_combined_canny: 80.0% variancia eléréséhez 1403 komponens szükséges.
all_combined_canny: 90.0% variancia eléréséhez 1954 komponens szükséges.
all_combined_canny: 95.0% variancia eléréséhez 2366 komponens szükséges.
all_combined_canny_variance_plot.png mentve.
feature_reduction\canny\all_combined\all_combined_canny_pca_10.csv mentve.
feature_reduction\canny\all_combined\all_combined_canny_pca_20.csv mentve.
feature_reduction\canny\all_combined\all_combined_canny_pca_50.csv mentve.
feature_reduction\canny\all_combined\all_combined_canny_pca_100.csv mentve.
feature_reduction\canny\all_combined\all_combined_canny_pca_200.csv mentve.
feature_reduction\canny\all_combined\all_combined_canny_pca_500.csv mentve.


Előfeldolgozási típusok: 100%|██████████| 5/5 [1:18:48<00:00, 945.65s/it]

feature_reduction\canny\all_combined\all_combined_canny_pca_1000.csv mentve.
